### An example notebook that calculates average hotel ratings for cities in Sweden and displays it on interactive kepler.gl map

In [ ]:
import pandas as pd
import datetime
from booking_scraper import bkscraper
from keplergl import KeplerGl
import math

### Read dataframe

In [ ]:
df = pd.read_csv('se.csv')

In [ ]:
df.head(10)

### Calculate average hotel ratings

In [ ]:
def calculate_average_hotel_rating(city):
    '''Calculates the average hotel rating given a city name in Sweden.
    Args:
        city: a city name in Sweden
    Returns:
        Average hotel rating if at least 5 hotels with scores is found, 0 otherwise
    '''
    
    today = datetime.datetime.now()
    start_date = today + datetime.timedelta(7)
    end_date = start_date + datetime.timedelta(7)
    result = bkscraper.get_result(city=city,  country='Sweden', datein=start_date, dateout=end_date, limit=-1, detail=False, is_verbose=False)
    scores = []
    for hotel in result:
        if hotel.get('score'):
            scores.append(float(hotel['score'].replace(',','.')))
    if len(scores) > 4:
        return sum(scores)/len(scores)
    return 0.0

In [ ]:
df['average_hotel_rating'] = df.city.apply(lambda x: calculate_average_hotel_rating(x))


In [ ]:
# Use exp in order to amplify the differences between average ratings 
df['exp_average_hotel_rating'] = df.average_hotel_rating.apply(lambda x : math.exp(x))

### FIlter the cities with no scores

In [ ]:
df = df[df.exp_average_hotel_rating > 1]

### Save the results for later use

In [ ]:
df.to_csv('sweden_hotel_ratings.csv', index=False)

In [ ]:
df = pd.read_csv('sweden_hotel_ratings.csv')

In [ ]:
df.head(5)

### Display the map

In [ ]:
from map_config import config

In [ ]:
# Load an empty map
map_1 = KeplerGl(height=500, data={'sweden_hotel_rating_data': df})
map_1.config = config
map_1

### Save the map

In [ ]:
map_1.save_to_html(data={'sweden_hotel_rating_data': df}, file_name='sweden_hotel_rating_map.html')
